In [1]:
from music21 import *
import os
import sys
import pandas as pd
import re,itertools
import site

In [2]:
site.getsitepackages()
#os.getcwd()
#Links before i loose them
#https://docs.google.com/viewer?a=v&pid=sites&srcid=ZGVmYXVsdGRvbWFpbnxtbDRtZDIwMTZ8Z3g6NTUwZThmNTcxODcxMWVmYQ
#http://musicweb.ucsd.edu/~sdubnov/Mu270d/Harmony/marc-steedman.pdf
#https://www.youtube.com/user/patdavidmusic/videos
#http://ai.stanford.edu/~kdtang/papers/cmj10-jazzgrammar.pdf
#http://www.iro.umontreal.ca/~pift6080/H08/documents/papers/lstm_music.pdf
#https://www.youtube.com/watch?v=oGGVvTgHMHw

['/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages',
 '/Library/Python/3.5/site-packages']

In [8]:
def _parse(file):
    midi = converter.parse(file)
    if midi is None:
        raise sys.exit('File path invalid')
    #depending where your melody is return appropriate index; using zero
    return midi[0]

    

In [41]:
def generateNotes(fileName):
    #parse
    noteList = []
    part = _parse(fileName)
    print(part.getElementsByClass(tempo.MetronomeMark)[0].number)
    print('{0}/{1}'.format(part.getElementsByClass(meter.TimeSignature)[0].numerator,part.getElementsByClass(meter.TimeSignature)[0].denominator))
    for i,v in enumerate(part.getElementsByClass(stream.Voice)):
        notes = v.getElementsByClass(note.Note).notes;
        for i in notes:
            noteList.append(i)
    if noteList:
        print("Note/Rest,Octave,Quarter,Offset")
        for n in noteList:
            print('{0},{1},{2},{3}'.format(n.name,n.octave,n.quarterLength,float(n.offset)))
            
    else: raise sys.exit("Note List Empty")       
    


def generateChords(fileName):
    #parse
    chordList = []
    part = _parse(fileName)
    print(part.getElementsByClass(tempo.MetronomeMark)[0].number)
    print('{0}/{1}'.format(part.getElementsByClass(meter.TimeSignature)[0].numerator,part.getElementsByClass(meter.TimeSignature)[0].denominator))
    for idx,v in enumerate(part.getElementsByClass(stream.Voice)):
        chords = v.getElementsByClass(chord.Chord)
        for c in chords:
            chordList.append(c)
            
    if chordList:
        print("FullName,CommonName,Length,Offset")
        for cl in chordList:
            print('{0},{1},{2},{3}'.format(cl.fullName,cl.pitchedCommonName,cl.quarterLength,float(cl.offset)))
    else: raise sys.exit("chord list empty")


In [75]:
# TODO Improve toggling
#if('notes' == sys.argv[1]):
#print('Generating notes for MIDI')
#generateNotes('/Users/Dixit/Documents/github/jazzml/Oscar-Peterson-2.mid')
#generateChords('/Users/Dixit/Documents/github/jazzml/Oscar-Peterson-2.mid')
#elif('chords' == sys.argv[1]):
#print('Generating Chords for MIDI')

#else :
    #raise sys.exit('Wrong Arg');

In [295]:
# The Chords generated are not unique; So need to extract and clean unique chords from above
#http://stackoverflow.com/questions/773/how-do-i-use-pythons-itertools-groupby
chords=[]
def readChordsFromFile():
    allChords = []
    with open('./midiToChords.txt','r') as f:
        mm = f.readline()
        sigNum,sigDen = [i for i in f.readline().split('/')]
    #Pandas csv reader
    chords = pd.read_csv('midiToChords.txt',sep=',',skiprows=2)[:]
    chords = chords.sort_values(by="Offset")
    return mm,sigNum,sigDen,chords



In [296]:
mm,sigNum,sigDen,chords=readChordsFromFile()
#os.getcwd()

In [297]:
print("mm,sigNum,sigDen,chords length")
print(mm,sigNum,sigDen,len(chords))

mm,sigNum,sigDen,chords length
176.0
 4 4
 297


In [299]:
def getUniqueChords(chords):
    sortedChords=[]
    for cname in chords['FullName']:
        #basic notes are C D E F G A B 
        #everythin is in string so regexp
        # '#' is sharp , '-' is flat in music21
        notes = re.findall('[CDEFGAB]+[-]*[flat|sharp]*[in octave]+[1-9]+',cname)
        for k in range(len(notes)):
            notes[k] = notes[k].replace(' in octave ','').replace('-sharp','#').replace('-flat','-')
        sortedChords.append(sorted(notes))
    return sortedChords
        

In [301]:
sortedChords = getUniqueChords(chords)
duplicate = sortedChords

In [302]:
finalResult=[]
for key,group in itertools.groupby(duplicate):
    finalResult.append(key)
finalResult = sorted(finalResult)    

In [303]:
finalResult2 =[]
for key2,group2 in itertools.groupby(finalResult):
    finalResult2.append(key2)

In [304]:
finalResult2Copy = finalResult2
print(len(finalResult2Copy))


228


In [316]:
for fr in finalResult2Copy:
    for c in fr:
        if '6' in c or '5' in c:
            fr.remove(c)
list2 = [x for x in finalResult2Copy if x]
list3 = [x for x in list2 if len(x)>2]
            


In [318]:
len(list3)

45

In [319]:
finalResult2=list3 


In [320]:
finalResult3 =[]
for key3,group3 in itertools.groupby(sorted(finalResult2)):
    finalResult3.append(key3)

In [321]:
#maybe best so far ; getting same length while using linux sort
len(finalResult3)

40

In [322]:
with open("uniqueChords.txt", 'w') as f:
    for chord in finalResult3:
        for n in chord:
            f.write(n)
            f.write(' ')
        f.write('\n')

In [212]:
#----------------------xxxxxxxxxxx------------------------------#